In [8]:
from utils import *
from datasets import load_dataset
def get_transform(image_size=256):
    transform = transforms.Compose([
        #transforms.Lambda(lambda pil_image: center_crop_arr(pil_image, image_size=image_size)),
        transforms.Resize((image_size, image_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5,], inplace=True)
    ])
    return transform

class ImageDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]['image'], self.dataset[idx]['label']
        image = self.transform(image).unsqueeze(0)
        #print(image.shape)
        return image, label
    
dataset = load_dataset("imagenet-1k",cache_dir='./dataset/imagenet1k')
transform = get_transform(image_size=256)
train_dataset = ImageDataset(dataset['train'], transform)
#validation_dataset = ImageDataset(dataset['validation'], transform)
test_dataset = ImageDataset(dataset['test'], transform)

batch_size=int(32)
train_sampler, train_loader = get_ddp_sampler_loader(dataset=train_dataset,
                            num_replicas=4,
                            rank=0,
                            sample_shuffle=True,
                            seed=0,
                            batch_size=batch_size,
                            num_workers=4,
                            pin_memory=True,
                            drop_last=True)

In [12]:
train_dataset[0][0].unsqueeze(0).shape

torch.Size([1, 3, 256, 256])

In [9]:
for batch in train_loader:
    print(batch[0].shape)
    break

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/AD/xiw136/anaconda3/envs/DiT/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/AD/xiw136/anaconda3/envs/DiT/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/AD/xiw136/anaconda3/envs/DiT/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/tmp/ipykernel_518880/2984199383.py", line 23, in __getitem__
    image = self.transform(image)
            ^^^^^^^^^^^^^^^^^^^^^
  File "/home/AD/xiw136/anaconda3/envs/DiT/lib/python3.11/site-packages/to